## IMPORT

In [1]:
!git clone https://github.com/giankev/PDLPR-algorithm.git

Cloning into 'PDLPR-algorithm'...
remote: Enumerating objects: 721, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 721 (delta 26), reused 71 (delta 18), pack-reused 638 (from 2)
Receiving objects: 100% (721/721), 146.48 MiB | 35.09 MiB/s, done.
Resolving deltas: 100% (286/286), done.


In [2]:
# standard library
import os
import sys
import math
import time
import shutil
import tarfile
import warnings
from pathlib import Path

# utility
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import time

#PyTorch & torchvision
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms as T
from torchvision.ops import box_iou
import torchvision.transforms as T

#Albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

#Custom repo modules 
repo_path = "/kaggle/working/PDLPR-algorithm/baseline_scr/detection"
sys.path.insert(0, repo_path)
from model import LPDetectorFPN
sys.path.remove(repo_path)

repo_path = "/kaggle/working/PDLPR-algorithm/baseline_scr/recognition"
sys.path.insert(0, repo_path)
from module import BaselineRecognizer
sys.path.remove(repo_path)

warnings.filterwarnings("ignore")

## SETUP ENVIRONMENT

In [3]:
#downloading 50k imgs for train and 8k for test
!gdown --folder https://drive.google.com/drive/folders/143HxhUrqkFIdfCzZQ3dA4Mqt8cjARCxx?usp=sharing -O datasets
#https://drive.google.com/drive/u/1/folders/1Qirh0lsjdsroLHEmJDtS6sVXPQKalW6j

Retrieving folder contents
Processing file 1rlOc7X2_C9vq2sm1ULBjNAgb_gy6CP8R ccpd_test.tar
Processing file 1hqZnTIOaRIaPPfN-juQKADCnE4ZJqqtO ccpd_train.tar
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1rlOc7X2_C9vq2sm1ULBjNAgb_gy6CP8R
From (redirected): https://drive.google.com/uc?id=1rlOc7X2_C9vq2sm1ULBjNAgb_gy6CP8R&confirm=t&uuid=218ca55b-a186-4c42-85db-6e09889b2494
To: /kaggle/working/datasets/ccpd_test.tar
100%|█████████████████████████████████████████| 557M/557M [00:02<00:00, 216MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1hqZnTIOaRIaPPfN-juQKADCnE4ZJqqtO
From (redirected): https://drive.google.com/uc?id=1hqZnTIOaRIaPPfN-juQKADCnE4ZJqqtO&confirm=t&uuid=a376aeb5-d6a6-4fe0-838d-557225c88ea3
To: /kaggle/working/datasets/ccpd_train.tar
100%|███████████████████████████████████████| 3.76G/3.76G [00:32<00:00, 115MB/s]
Download completed


In [4]:
!gdown --fuzzy https://drive.google.com/file/d/1t0d9yFnCPztuQVm_l7CbXkx2NM9tCif6/view?usp=drive_link 

Downloading...
From (original): https://drive.google.com/uc?id=1t0d9yFnCPztuQVm_l7CbXkx2NM9tCif6
From (redirected): https://drive.google.com/uc?id=1t0d9yFnCPztuQVm_l7CbXkx2NM9tCif6&confirm=t&uuid=36cb27bf-dbb9-40f1-8b30-78863491ca98
To: /kaggle/working/base_rec_checkpoint_epoch50.pt
100%|█████████████████████████████████████████| 264M/264M [00:01<00:00, 209MB/s]


In [5]:
# extracting the .tar archive.
def extract_tar_archive(archive_path, destination_path):

    print(f"Extracting the tar archive in:{archive_path}")
    with tarfile.open(archive_path, "r") as tar:
        tar.extractall(path=destination_path)
        
    print(f"Archive extracted in: {destination_path}")

#delete the .tar archive which now is useless.
def delete_tar_archive(path_tar_archive):
    
    if os.path.exists(path_tar_archive):
        shutil.rmtree(path_tar_archive)
        print(f"Folder eliminated: {path_tar_archive}")
    else:
        print(f"Folder not found: {path_tar_archive}")

In [6]:
archive_path_train = "/kaggle/working/datasets/ccpd_train.tar"
archive_path_test = "/kaggle/working/datasets/ccpd_test.tar"
extract_path = "/kaggle/working/"

#when extracting the files, is important to eliminate the .tar archive which now occupy /kaggle/working space.
extract_tar_archive(archive_path_train, extract_path)
extract_tar_archive(archive_path_test, extract_path)
delete_tar_archive("/kaggle/working/datasets/")

Extracting the tar archive in:/kaggle/working/datasets/ccpd_train.tar
Archive extracted in: /kaggle/working/
Extracting the tar archive in:/kaggle/working/datasets/ccpd_test.tar
Archive extracted in: /kaggle/working/
Folder eliminated: /kaggle/working/datasets/


In [7]:
PATH_WEIGHTS_DETECTION = "/kaggle/working/PDLPR-algorithm/baseline_scr/detection/detec_weights.pt"
PATH_WEIGHTS_RECOGNITION = "/kaggle/working/base_rec_checkpoint_epoch50.pt"
TEST_ROOT = "/kaggle/working/ccpd_test"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model_det = LPDetectorFPN()
state_dict = torch.load(PATH_WEIGHTS_DETECTION, map_location="cpu") 
model_det.load_state_dict(state_dict)

model_det.to(device).eval()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 202MB/s]


LPDetectorFPN(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [9]:
model_rec = BaselineRecognizer()
state_dict = torch.load(PATH_WEIGHTS_RECOGNITION, map_location="cpu") 
model_rec.load_state_dict(state_dict["weights"])

model_rec.to(device).eval()

BaselineRecognizer(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
  )
  (rnn): LSTM(3072, 512, num_layers=2, batch_first=True,

In [10]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣",
             "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]

alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N',
             'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']

ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R',
       'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

unique_chars = set(provinces[:-1] + alphabets[:-1] + ads[:-1])  # escludi 'O'
char_list = sorted(list(unique_chars))  # ordinamento per coerenza
char_list = ["-"] + char_list
char2idx = {char: i for i, char in enumerate(char_list)}
idx2char = {i: c for c, i in char2idx.items()}

num_classes = len(char_list)
print("Num classes: ", num_classes)

Num classes:  68


In [11]:
#extracting the metadata from each img in this format (image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number)
def decode_plate(s):
    "this method is used for decoding the plate starting from the name of .jpg file"
    idx   = list(map(int, s.split("_")))
    try:
        return provinces[idx[0]] + alphabets[idx[1]] + "".join(ads[i] for i in idx[2:])
    except Exception:
        return None

#extracting the metadata from each img in this format (image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox)
def split_bbox(bbox_str):
    "extracting x1,y1,x2,y2, ex. '283___502_511___591'  →  ['283','502','511','591']"
    tokens = []
    for seg in bbox_str.split("___"):
        tokens.extend(seg.split("_"))
    if len(tokens) == 4 and all(t.isdigit() for t in tokens):
        return map(int, tokens)
    return (None,)*4

## TEST PHASE

In [12]:
folder = "/kaggle/working/ccpd_test"
rows = []

for root, _, files in os.walk(folder):
    for fname in files:
        if not fname.endswith(".jpg"):
            continue

        parts = fname[:-4].split("-")
        if len(parts) < 6:
            continue

        x1, y1, x2, y2 = split_bbox(parts[2])
        plate = decode_plate(parts[4])
        full_path = os.path.join(root, fname)

        rows.append({
            "image_path": full_path,
            "x1_bbox": x1,
            "y1_bbox": y1,
            "x2_bbox": x2,
            "y2_bbox": y2,
            "plate_number": plate
        })

df = pd.DataFrame(rows)
df["subset"] = df["image_path"].apply(lambda p: Path(p).parts[-2])
print(f"Dataset created: {len(df)} rows")
df.head()

Dataset created: 8000 rows


,image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number,subset
0,/kaggle/working/ccpd_test/fn/0777-6_20-113___4...,113,469,529,625,皖AY165D,fn
1,/kaggle/working/ccpd_test/fn/2067-3_2-0___321_...,0,321,688,572,皖AS276E,fn
2,/kaggle/working/ccpd_test/fn/0068-3_4-198___47...,198,471,299,528,皖AX868C,fn
3,/kaggle/working/ccpd_test/fn/0836-4_3-111___44...,111,446,553,604,皖A0X569,fn
4,/kaggle/working/ccpd_test/fn/0675-18_28-194___...,194,356,482,552,皖AT0581,fn


In [15]:
# CONFIG
IOU_THR      = 0.60          # detection is accepted if IoU ≥ 0.60
DET_IN       = 224           # detector input side
REC_H, REC_W = 48, 144       # recogniser input (HxW)
BATCH_SIZE   = 5             # dataloader batch for detector


tf_det = T.Compose([
    T.ToPILImage(),
    T.Resize((DET_IN, DET_IN), interpolation=T.InterpolationMode.BILINEAR),
    T.ToTensor()
])

def collate_fn(batch):
    det_batch   = torch.stack([b[0] for b in batch])   # B×3×224×224
    rgb_list    = [b[1] for b in batch]
    bbox_list   = [b[2] for b in batch]                # list of tensors
    plate_list  = [b[3] for b in batch]
    hw_list     = [b[4] for b in batch]
    return det_batch, rgb_list, bbox_list, plate_list, hw_list

loader = DataLoader(CCPDTestDataset(df),
                    batch_size=BATCH_SIZE,
                    shuffle=False, num_workers=4,
                    collate_fn=collate_fn, pin_memory=True)


# HELPERS
idx2char = {i: c for c, i in char2idx.items()}

def logits_to_plate(logits: torch.Tensor) -> str:
    return ''.join(idx2char[i] for i in logits.argmax(-1).tolist())

def xyxy_iou(a, b):                      # tensors length‑4
    ix = (min(a[2], b[2]) - max(a[0], b[0])).clamp_(0)
    iy = (min(a[3], b[3]) - max(a[1], b[1])).clamp_(0)
    inter = ix * iy
    area_a = (a[2]-a[0])*(a[3]-a[1])
    area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-7)

tf_rec = T.Compose([
    T.ToPILImage(),
    T.Resize((REC_H, REC_W), interpolation=T.InterpolationMode.BILINEAR),
    T.ToTensor()
])

In [16]:
class CCPDTestDataset(torch.utils.data.Dataset):
    """Returns:
       det_tensor : 3×224×224   (for detector)
       rgb_orig   : H×W×3  (np.uint8) original image for cropping
       bbox_gt    : tensor[4]   (x1,y1,x2,y2) ground‑truth
       plate_str  : str         ground‑truth plate
       orig_hw    : (H,W)       original size
    """
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

    def __len__(self):  return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        rgb = cv2.cvtColor(cv2.imread(row.image_path), cv2.COLOR_BGR2RGB)
        H, W = rgb.shape[:2]

        det_tensor = tf_det(rgb)          # torch tensor 3×224×224
        bbox_gt = torch.tensor([row.x1_bbox, row.y1_bbox,
                                row.x2_bbox, row.y2_bbox], dtype=torch.float32)
        return det_tensor, rgb, bbox_gt, row.plate_number, (H, W)

In [17]:
df["subset"] = df.image_path.apply(lambda p: Path(p).parent.name)
results = []

for subset in sorted(df['subset'].unique()):
    print(f"\nEvaluating subset: {subset}")
    df_subset = df[df.subset == subset].copy()

    # DataLoader
    loader = DataLoader(CCPDTestDataset(df_subset),
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        num_workers=4,
                        collate_fn=collate_fn,
                        pin_memory=True)

    total_imgs = correct_plates = 0
    t_start = time.perf_counter()

    for det_batch, rgb_list, bbox_list, plate_gt_list, hw_list in loader:
        B = det_batch.size(0)

        # Detector forward
        with torch.no_grad():
            preds = model_det(det_batch.to(device)).cpu()  # Bx4

        # Iterate over batch
        for i in range(B):
            total_imgs += 1
            H0, W0 = hw_list[i]
            cx, cy, w, h = (preds[i] * DET_IN).tolist()

            # Back-project to original image size
            x1 = (cx - w/2) * W0 / DET_IN
            y1 = (cy - h/2) * H0 / DET_IN
            x2 = (cx + w/2) * W0 / DET_IN
            y2 = (cy + h/2) * H0 / DET_IN
            pred_xy = torch.tensor([x1, y1, x2, y2])

            iou = xyxy_iou(pred_xy, bbox_list[i])
            if iou < IOU_THR:
                continue  # detection failed

            # Crop & recognise
            x1i, y1i, x2i, y2i = map(int, [max(0,x1), max(0,y1), min(W0,x2), min(H0,y2)])
            if x2i <= x1i or y2i <= y1i:
                continue

            crop_rgb = rgb_list[i][y1i:y2i, x1i:x2i]
            rec_in = tf_rec(crop_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                logits = model_rec(rec_in)[0].cpu()

            if logits_to_plate(logits) == plate_gt_list[i]:
                correct_plates += 1

    t_end = time.perf_counter()
    fps = total_imgs / (t_end - t_start)
    acc = correct_plates / total_imgs if total_imgs else 0.0

    print(f"Subset: {subset:12} | Accuracy: {acc:.3f} | FPS: {fps:.1f}")
    results.append((subset, acc, fps))

# Summary table
print("\n" + "-"*40)
print(f"{'Subset':12} | {'Acc':>7} | {'FPS':>6}")
print("-"*40)
for s, a, f in results:
    print(f"{s:12} | {a:7.3f} | {f:6.1f}")


Evaluating subset: base
Subset: base         | Accuracy: 0.997 | FPS: 87.7

Evaluating subset: blur
Subset: blur         | Accuracy: 0.777 | FPS: 88.5

Evaluating subset: challenge
Subset: challenge    | Accuracy: 0.826 | FPS: 90.3

Evaluating subset: db
Subset: db           | Accuracy: 0.764 | FPS: 90.3

Evaluating subset: fn
Subset: fn           | Accuracy: 0.800 | FPS: 84.6

Evaluating subset: rotate
Subset: rotate       | Accuracy: 0.933 | FPS: 80.7

Evaluating subset: tilt
Subset: tilt         | Accuracy: 0.865 | FPS: 81.3

Evaluating subset: weather
Subset: weather      | Accuracy: 0.987 | FPS: 84.3

----------------------------------------
Subset       |     Acc |    FPS
----------------------------------------
base         |   0.997 |   87.7
blur         |   0.777 |   88.5
challenge    |   0.826 |   90.3
db           |   0.764 |   90.3
fn           |   0.800 |   84.6
rotate       |   0.933 |   80.7
tilt         |   0.865 |   81.3
weather      |   0.987 |   84.3
